In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
import joblib


In [9]:
file_path = "../data/predictive_maintenance.csv"  # Substitua pelo caminho correto do arquivo
data = pd.read_csv(file_path)

# Selecionar as features e processar Material L e Material M
data['Material L'] = (data['Type'] == 'L').astype(int)
data['Material M'] = (data['Type'] == 'M').astype(int)

features = data[[
    "Air temperature [K]", 
    "Process temperature [K]", 
    "Rotational speed [rpm]", 
    "Torque [Nm]", 
    "Tool wear [min]",
    "Material L",
    "Material M"
]]

# Codificar as classes de saída (Failure Type)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data["Failure Type"])

# Normalizar as features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    scaled_features, labels, test_size=0.2, random_state=42
)


In [10]:
# Criar o modelo da rede neural com quatro camadas ocultas
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')  # Saída com 6 classes
])

/home/pgm6/anaconda3/envs/mlops/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-19 18:20:57.923396: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [11]:
# Compilar o modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Treinar o modelo
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6844 - loss: 0.9928 - val_accuracy: 0.9594 - val_loss: 0.2498
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9634 - loss: 0.3055 - val_accuracy: 0.9594 - val_loss: 0.1894
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9610 - loss: 0.2368 - val_accuracy: 0.9594 - val_loss: 0.1794
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9634 - loss: 0.1850 - val_accuracy: 0.9594 - val_loss: 0.1716
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9682 - loss: 0.1667 - val_accuracy: 0.9594 - val_loss: 0.1679
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9661 - loss: 0.1590 - val_accuracy: 0.9594 - val_loss: 0.1665
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9674 - loss: 0.1620 - val_accuracy: 0.9594 - val_loss: 0.1576
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9666 - loss: 0.1537 - val_accuracy: 0.

In [13]:
# Avaliar o modelo no conjunto de teste
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print("Loss no teste:", test_loss)
print("Acurácia no teste:", test_accuracy)


Loss no teste: 0.09954789280891418
Acurácia no teste: 0.9729999899864197


In [15]:
joblib.dump(label_encoder, "../models/label_encoder.pkl")
joblib.dump(scaler, "../models/scaler.pkl")
model.save("../models/pm_classificacao.h5")

In [9]:
from tensorflow.keras.models import load_model
import numpy as np

In [10]:
model = load_model("scr/pm_classificacao.h5")

In [11]:
scaler = joblib.load("scr/scaler.pkl")

new_sample = np.array([[300.1, 310.2, 1500, 50, 10, 1, 0]])

new_sample_scaled = scaler.transform(new_sample)

/home/pgm6/anaconda3/envs/mlops/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [12]:
# Fazer a predição
predictions = model.predict(new_sample_scaled)[0]

# Obter a classe com maior probabilidade
predicted_class = np.argmax(predictions)
predicted_label = label_encoder.inverse_transform([predicted_class])[0]

# Exibir as probabilidades para cada classe
print("Probabilidades por classe:")
for class_idx, probability in enumerate(predictions):
    label = label_encoder.inverse_transform([class_idx])[0]
    print(f"Classe: {class_idx}, Rótulo: {label}, Probabilidade: {probability:.4f}")

# Exibir o resultado final
print("\nClasse predita (índice):", predicted_class)
print("Rótulo predito:", predicted_label)
print("Probabilidade predita:", predictions[predicted_class])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Probabilidades por classe:
Classe: 0, Rótulo: Heat Dissipation Failure, Probabilidade: 0.0001
Classe: 1, Rótulo: No Failure, Probabilidade: 0.9975
Classe: 2, Rótulo: Overstrain Failure, Probabilidade: 0.0000
Classe: 3, Rótulo: Power Failure, Probabilidade: 0.0001
Classe: 4, Rótulo: Random Failures, Probabilidade: 0.0018
Classe: 5, Rótulo: Tool Wear Failure, Probabilidade: 0.0003

Classe predita (índice): 1
Rótulo predito: No Failure
Probabilidade predita: 0.9975406
